In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import imageio

import os
from math import * 
os.getcwd()
print("The current directory is: ")
print(os.getcwd())
path_parent = os.path.dirname(os.getcwd())
if path_parent.endswith("Abaqus-Nanoindentation-Project"):
    os.chdir(path_parent) 
    print("Moving up to the parent directory")
    print(os.getcwd())

The current directory is: 
c:\Users\springnuance\Desktop\Abaqus-Nanoindentation-Project\optimizers
Moving up to the parent directory
c:\Users\springnuance\Desktop\Abaqus-Nanoindentation-Project


In [2]:
grains = ["1441","2744","3198"]
strainRates = ["0p1"]
CPLaw = "PH"
material = "ferrite_n20"

# The project path folder
projectPath = os.getcwd()

# The logging path
logPath = f"log/{CPLaw}_{material}.txt"
# The paramInfo path
paramInfoPath = f"paramInfo/{CPLaw}_{material}"
# The results path
resultPath = f"results/{CPLaw}_{material}"
# The simulations path
simPath = f"simulations/{CPLaw}_{material}"
# The target path
targetPath = f"targets/{CPLaw}_{material}"
# The templates path
templatePath = f"templates/{CPLaw}_{material}"

In [14]:
import copy

objectives = [
    "grain_1441_sr_0p1",
    "grain_2744_sr_0p1",
    "grain_3198_sr_0p1"
]

# Loading initial simulations
exampleObjective = objectives[0]

initial_objective_value_to_param_FD_Curves = {}

for objective in objectives:
    initial_objective_value_to_param_FD_Curves[objective] = np.load(f"{resultPath}/{objective}/initial/common/FD_Curves.npy", allow_pickle=True).tolist()

iteration_objective_value_to_param_FD_Curves = {}

# Check if there are any iteration simulations
if not os.path.exists(f"{resultPath}/{exampleObjective}/iteration/common/FD_Curves.npy"):
    print("There are no iteration simulations")
    for objective in objectives:
        iteration_objective_value_to_param_FD_Curves[objective] = {}
else:
    print("Iteration simulations exist", logPath)
    numberOfIterationSims = len(np.load(f"{resultPath}/{exampleObjective}/iteration/common/FD_Curves.npy", allow_pickle=True).tolist())
    print(f"Number of iteration simulations: {numberOfIterationSims} FD curves")
    for objective in objectives:
        iteration_objective_value_to_param_FD_Curves[objective] = np.load(f"{resultPath}/{objective}/iteration/common/FD_Curves.npy", allow_pickle=True).tolist()

combined_objective_value_to_param_FD_Curves = copy.deepcopy(initial_objective_value_to_param_FD_Curves)

for objective in objectives:
    combined_objective_value_to_param_FD_Curves[objective].update(iteration_objective_value_to_param_FD_Curves[objective])


nonconverging_objectives_to_params = {}
for objective in objectives:    
    nonconverging_objectives_to_params[objective] = {}
    nonconverging_objectives_to_params[objective]["paramFeatures"] = []
    nonconverging_objectives_to_params[objective]["nonconvergingLabels"] = []
    
    for params, dispForce in combined_objective_value_to_param_FD_Curves[objective].items():
        paramsList = [paramValue for paramName, paramValue in params]
        nonconverging_objectives_to_params[objective]["paramFeatures"].append(paramsList)
        
        force = dispForce["force"]
        if len(force) < 1000:
            nonconverging_objectives_to_params[objective]["nonconvergingLabels"].append(1)
        else: 
            nonconverging_objectives_to_params[objective]["nonconvergingLabels"].append(0)

# Convert to numpy 

for objective in objectives:
    nonconverging_objectives_to_params[objective]["paramFeatures"] = np.array(nonconverging_objectives_to_params[objective]["paramFeatures"])
    nonconverging_objectives_to_params[objective]["nonconvergingLabels"] = np.array(nonconverging_objectives_to_params[objective]["nonconvergingLabels"])        
    print(nonconverging_objectives_to_params[objective]["paramFeatures"].shape)
    print(nonconverging_objectives_to_params[objective]["nonconvergingLabels"].shape)

There are no iteration simulations
(20, 4)
(20,)
(20, 4)
(20,)
(20, 4)
(20,)


In [15]:
print("Nonconverging objectives to params") 
print(nonconverging_objectives_to_params)

Nonconverging objectives to params
{'grain_1441_sr_0p1': {'paramFeatures': array([[3.00000e+02, 1.30000e+00, 4.50000e+03, 3.70000e+02],
       [4.00000e+02, 9.00000e-01, 5.25000e+03, 3.20000e+02],
       [2.00000e+02, 1.70000e+00, 3.75000e+03, 4.20000e+02],
       [2.50000e+02, 1.10000e+00, 4.87500e+03, 2.95000e+02],
       [4.50000e+02, 1.90000e+00, 3.37500e+03, 3.95000e+02],
       [3.50000e+02, 7.00000e-01, 4.12500e+03, 3.45000e+02],
       [1.50000e+02, 1.50000e+00, 5.62500e+03, 4.45000e+02],
       [1.75000e+02, 1.00000e+00, 3.93750e+03, 4.07500e+02],
       [3.75000e+02, 1.80000e+00, 5.43750e+03, 3.07500e+02],
       [4.75000e+02, 6.00000e-01, 4.68750e+03, 4.57500e+02],
       [2.75000e+02, 1.40000e+00, 3.18750e+03, 3.57500e+02],
       [2.25000e+02, 8.00000e-01, 5.81250e+03, 3.82500e+02],
       [4.25000e+02, 1.60000e+00, 4.31250e+03, 2.82500e+02],
       [3.25000e+02, 1.20000e+00, 3.56250e+03, 4.32500e+02],
       [1.25000e+02, 2.00000e+00, 5.06250e+03, 3.32500e+02],
       [1.

In [25]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


np.random.seed(42) # For reproducibility

classifiers = {}

for objective in objectives:
    X = nonconverging_objectives_to_params[objective]["paramFeatures"]  # Features
    y = nonconverging_objectives_to_params[objective]["nonconvergingLabels"]  # Labels

    # Creating the SVM classifier
    clf = SVC(C=1.0, kernel='linear')  # Using a linear kernel

    # Training the classifier
    clf.fit(X, y)

    # Making predictions
    y_pred = clf.predict(X)
    
    classifiers[objective] = clf
    # Evaluating the classifier
    accuracy = accuracy_score(y, y_pred)

    print(f"Accuracy for {objective}: {accuracy*100:.2f}%")

Accuracy for grain_1441_sr_0p1: 100.00%
Accuracy for grain_2744_sr_0p1: 100.00%
Accuracy for grain_3198_sr_0p1: 95.00%
